### HW
1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии
- Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)
- Ссылку на саму вакансию
- Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.


In [13]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs

pd.set_option('display.max_rows',500)

### HeadHunter

In [14]:
# Исходные данные
main_link = 'https://hh.ru/search/vacancy'
vacansy = 'Data scientist'
header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36'}
param = {
    'area': 1,
    'st': 'searchVacancy',
    'text': vacansy
}

In [15]:
df_hh = pd.DataFrame()

i = 0
j = 0
while True:
    # Переход по страницам
    param['page'] = j
    j += 1
    response = requests.get(main_link, headers=header, params=param).text
    soup = bs(response,'lxml')
    
    # Наименование вакансии
    res_vacansy = soup.find_all('a', attrs={'class' : "bloko-link HH-LinkModifier"})
    res_vacansy_list = [i.getText() for i in res_vacansy]
    
    # Предлагаемая зарплату (отдельно мин. отдельно макс. и отдельно валюта)
    res_salary = soup.find_all('div', attrs={'class' :"vacancy-serp-item__row_header"})
    res_salary_list = [i.find('div', attrs={'class' :"vacancy-serp-item__sidebar"}).getText() for i in res_salary]
    
    res_salary_min = ['н/у' for i in range(0,len(res_salary_list))]
    res_salary_max = ['н/у' for i in range(0,len(res_salary_list))]
    res_salary_currency = ['н/у' for i in range(0,len(res_salary_list))]
    
    for i in range(0, len(res_salary_list)):
        if 'от' in res_salary_list[i]:
            res_salary_min[i] = res_salary_list[i]
        if 'до' in res_salary_list[i]:
            res_salary_max[i] = res_salary_list[i]
        if ('EUR' or 'USD') in res_salary_list[i]:
            res_salary_currency[i] = res_salary_list[i]
        if '-' in res_salary_list[i]:
            res_salary_min[i] = res_salary_list[i][:res_salary_list[i].find('-')]
            res_salary_max[i] = res_salary_list[i][res_salary_list[i].find('-')+1:]
                    
    # Ссылку на саму вакансию
    res_link = soup.find_all('a', attrs={'class' : "bloko-link HH-LinkModifier"})
    res_link_list = [i['href'] for i in res_link]
    
    # Сайт откуда собрана вакансия
    res_site_list = ['HH' for i in range(0,len(res_link_list))]
    
    #Оформление в dataframe
    df1=pd.DataFrame(res_vacansy_list, columns=['vacancy'])
    df2=pd.DataFrame(res_salary_list, columns=['salary'])
    df21=pd.DataFrame(res_salary_min, columns=['min_salary'])
    df22=pd.DataFrame(res_salary_max, columns=['max_salary'])
    df23=pd.DataFrame(res_salary_currency, columns=['cur_salary'])
    df3=pd.DataFrame(res_link_list, columns=['link'])
    df4=pd.DataFrame(res_site_list, columns=['site'])

    df7=pd.concat([df1,df2,df21,df22,df23,df3,df4], axis=1)
       
    df_hh = pd.concat([df_hh, df7], axis=0, ignore_index=True)
        
    # Если кнопка дальше отсутствует - прекращаем парсинг
    if soup.find('a', {'class': 'HH-Pager-Controls-Next'}) == None:
        break
df_hh.to_csv("hh.csv")

In [16]:
df_hh

,vacancy,salary,min_salary,max_salary,cur_salary,link,site
0,Технический директор разработки SAAS продукта,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/37249572?query=Data%...,HH
1,Python Data Scientist,от 150 000 руб.,от 150 000 руб.,н/у,н/у,https://spb.hh.ru/vacancy/36916080?query=Data%...,HH
2,Senior Data Scientist,от 300 000 руб.,от 300 000 руб.,н/у,н/у,https://spb.hh.ru/vacancy/37294434?query=Data%...,HH
3,Data Scientist,от 130 000 руб.,от 130 000 руб.,н/у,н/у,https://spb.hh.ru/vacancy/37054908?query=Data%...,HH
4,Аналитик - эконометрист / Data scientist (Juni...,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/37172711?query=Data%...,HH
5,Data scientist,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/37292440?query=Data%...,HH
6,Middle/Senior data scientist (fraud prevention),140 000-220 000 руб.,140 000,220 000 руб.,н/у,https://spb.hh.ru/vacancy/36991107?query=Data%...,HH
7,Data Scientist,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/36452970?query=Data%...,HH
8,Data scientist,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/36877389?query=Data%...,HH
9,Data scientist,,н/у,н/у,н/у,https://spb.hh.ru/vacancy/37247506?query=Data%...,HH


### Super Job

In [17]:
# Исходные данные

main_link_sj = 'https://superjob.ru'
vacancy = 'it'
header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36'}
param = {
    'keywords':vacancy
}

In [18]:
df_sj = pd.DataFrame()

k = 0
while True:
    # Переход по страницам
    param['page'] = k
    k += 1
    response = requests.get(f'{main_link_sj}/vacancy/search/', headers=header, params=param).text
    soup = bs(response,'lxml')
    
    # Наименование вакансии
    res_vacansy = soup.find_all('div', attrs={'class' : "_3mfro PlM3e _2JVkc _3LJqf"})
    res_vacansy_list = [i.getText() for i in res_vacansy]
        
    # Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)
    res_salary = soup.find_all('span', attrs={'class' :"_3mfro _2Wp8I _1qw9T f-test-text-company-item-salary PlM3e _2JVkc _2VHxz"})
    res_salary_list = [i.getText() for i in res_salary]
    
    res_salary_min = ['н/у' for i in range(0,len(res_salary_list))]
    res_salary_max = ['н/у' for i in range(0,len(res_salary_list))]
    res_salary_currency = ['н/у' for i in range(0,len(res_salary_list))]
        
    for i in range(0, len(res_salary_list)):
        if 'от' in res_salary_list[i]:
            res_salary_min[i] = res_salary_list[i]
        elif 'до ' in res_salary_list[i]:
            res_salary_max[i] = res_salary_list[i]
        elif ('EUR' or 'USD') in res_salary_list[i]:
            res_salary_currency[i] = res_salary_list[i]
        elif '—' in res_salary_list[i]:
            res_salary_min[i] = res_salary_list[i][:res_salary_list[i].find('—')]
            res_salary_max[i] = res_salary_list[i][res_salary_list[i].find('—')+1:]
        else:
            res_salary_min[i] = res_salary_list[i]
            res_salary_max[i] = res_salary_list[i]
                    
    # Ссылку на саму вакансию
    res_link = soup.find_all('div', attrs={'class' : "_3mfro PlM3e _2JVkc _3LJqf"})
    res_link_list = [i.find('a')['href'] for i in res_link]
    res_link_list = [f'{main_link_sj}{i}' for i in res_link_list]
    res_link_list
    
    # Сайт откуда собрана вакансия
    res_site_list = ['SJ' for i in range(0, len(res_vacansy_list))]
    len(res_site_list)
    
    #Оформление в dataframe
    df1=pd.DataFrame(res_vacansy_list, columns=['vacancy'])
    df2=pd.DataFrame(res_salary_list, columns=['salary'])
    df21=pd.DataFrame(res_salary_min, columns=['min_salary'])
    df22=pd.DataFrame(res_salary_max, columns=['max_salary'])
    df23=pd.DataFrame(res_salary_currency, columns=['cur_salary'])
    df3=pd.DataFrame(res_link_list, columns=['link'])
    df4=pd.DataFrame(res_site_list, columns=['site'])

    df7=pd.concat([df1,df2,df21,df22,df23,df3,df4], axis=1)
       
    df_sj = pd.concat([df_sj, df7], axis=0, ignore_index=True)
        
    # Если кнопка дальше отсутствует - прекращаем парсинг
    if not soup.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'}):
        break
df_sj.to_csv("sj.csv")

In [19]:
df_sj

,vacancy,salary,min_salary,max_salary,cur_salary,link,site
0,"IT-специалист, программист 1 С",По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/it-specialist-339...,SJ
1,"Директор ИТ, Руководитель ИТ отдела, подраздел...",от 60 000 руб.,от 60 000 руб.,н/у,н/у,https://superjob.ru/vakansii/direktor-it-33915...,SJ
2,Руководитель IT-проектов,По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/rukovoditel-it-pr...,SJ
3,Заместитель директора по IT,По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/zamestitel-direkt...,SJ
4,"Системный администратор, IT-специалист",25 000 — 30 000 руб.,25 000,30 000 руб.,н/у,https://superjob.ru/vakansii/sistemnyj-adminis...,SJ
5,"Помощник системного администратора, IT-специалист",32 500 руб.,32 500 руб.,32 500 руб.,н/у,https://superjob.ru/vakansii/pomoschnik-sistem...,SJ
6,Инженер по IT,25 000 — 30 000 руб.,25 000,30 000 руб.,н/у,https://superjob.ru/vakansii/inzhener-po-it-33...,SJ
7,"IT-специалист, системный администратор",По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/it-specialist-332...,SJ
8,Бизнес-аналитик (IT),По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/biznes-analitik-3...,SJ
9,Заместитель директора по ИТ / Учитель информац...,По договорённости,По договорённости,По договорённости,н/у,https://superjob.ru/vakansii/zamestitel-direkt...,SJ
